<a href="https://colab.research.google.com/github/jalehsoltanpour/hello-world/blob/master/CRC_Qlearning_01_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import mdptoolbox, mdptoolbox.example
import pandas as pd
import numpy as np
import cmath
from scipy.stats.stats import pearsonr
from scipy.stats import poisson
from six.moves import cPickle as pickle
#import plotly.plotly as py
#import plotly.tools as tls
#import matplotlib.pyplot as plt
import math
import time
from scipy.optimize import linprog
from copy import deepcopy
import copy
from scipy.stats import norm
import decimal as ds
from itertools import permutations 
from itertools import combinations_with_replacement 
from random import sample 
import random
from scipy.stats import poisson
from scipy.stats import chi2
from random import choices
!python --version

Python 3.7.13


In [ ]:
states ={}
states[0] = 0
ct = 0
for k in range(7):
    for i1 in range(7):
        for i2 in range(7):
            for i3 in range(7):
                if ( (i1 + i2 + i3) == k and (i1 + i2 + i3)>=1 ):
                    ct +=1
                    state=(i3,i2,i1)
                    #print(state, ct)
                    states[state] = ct
#states ['7+'] = ct + 1
states ['E_CRC'] = ct + 1
states ['R_CRC'] = ct + 2
states ['A_CRC'] = ct + 3
states ['Tr_E'] = ct + 4
states ['Tr_R'] = ct + 5
states ['Tr_A'] = ct + 6
states ['D'] = ct + 7
len(states)


91

In [ ]:
states
### state 0: healthy
#### states 1:83 polyp states (num of small, num of medium, num of large)
#### states 84:86 cancer states
#### states 85:88 under treatment for different cancer states
### state 90: CRC state

{(0, 0, 1): 3,
 (0, 0, 2): 9,
 (0, 0, 3): 19,
 (0, 0, 4): 34,
 (0, 0, 5): 55,
 (0, 0, 6): 83,
 (0, 1, 0): 2,
 (0, 1, 1): 8,
 (0, 1, 2): 18,
 (0, 1, 3): 33,
 (0, 1, 4): 54,
 (0, 1, 5): 82,
 (0, 2, 0): 6,
 (0, 2, 1): 16,
 (0, 2, 2): 31,
 (0, 2, 3): 52,
 (0, 2, 4): 80,
 (0, 3, 0): 13,
 (0, 3, 1): 28,
 (0, 3, 2): 49,
 (0, 3, 3): 77,
 (0, 4, 0): 24,
 (0, 4, 1): 45,
 (0, 4, 2): 73,
 (0, 5, 0): 40,
 (0, 5, 1): 68,
 (0, 6, 0): 62,
 (1, 0, 0): 1,
 (1, 0, 1): 7,
 (1, 0, 2): 17,
 (1, 0, 3): 32,
 (1, 0, 4): 53,
 (1, 0, 5): 81,
 (1, 1, 0): 5,
 (1, 1, 1): 15,
 (1, 1, 2): 30,
 (1, 1, 3): 51,
 (1, 1, 4): 79,
 (1, 2, 0): 12,
 (1, 2, 1): 27,
 (1, 2, 2): 48,
 (1, 2, 3): 76,
 (1, 3, 0): 23,
 (1, 3, 1): 44,
 (1, 3, 2): 72,
 (1, 4, 0): 39,
 (1, 4, 1): 67,
 (1, 5, 0): 61,
 (2, 0, 0): 4,
 (2, 0, 1): 14,
 (2, 0, 2): 29,
 (2, 0, 3): 50,
 (2, 0, 4): 78,
 (2, 1, 0): 11,
 (2, 1, 1): 26,
 (2, 1, 2): 47,
 (2, 1, 3): 75,
 (2, 2, 0): 22,
 (2, 2, 1): 43,
 (2, 2, 2): 71,
 (2, 3, 0): 38,
 (2, 3, 1): 66,
 (2, 4, 0): 60,
 

In [ ]:
key_list = list(states.keys())
# index
val_list = list(states.values())
 
# print key with val st_num
st_num = 10
position = val_list.index(st_num)
key_list[position][2]


0

In [ ]:
Polyp_states = range(0,84)
Polyp_states

range(0, 84)

In [ ]:
CRC_states = [states['E_CRC'], states['R_CRC'], states['A_CRC']]
CRC_states

[84, 85, 86]

In [ ]:
partial_obs_st = range(0,87)

In [ ]:
Tr_state = [states['Tr_E'],states['Tr_R'],states['Tr_A']]
Tr_state

[87, 88, 89]

In [ ]:
Death_state = states['D']
Death_state

90

In [ ]:
n_states = len(states)
actions = [0,1]
n_action = len(actions)
T = 25 

# TPM for action W (Wait)

In [ ]:
P1 = pd.read_csv("psiNew_20.csv")

P = P1.values
P[82,:]

array([8.30000000e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [ ]:
### ref: https://www.nature.com/articles/6604788/tables/7
Large_adenoma_to_early = 0.05
Early_to_regional = 0.28
Regional_to_advanced = 0.35
Death_from_early = 0.0542
Death_from_regional = 0.1677
Death_from_advanced = 0.6469

In [ ]:
#ref: Amir code...  general death rates
GDR = np.array([5498,419,283,210,183,151,131,114,98,83,75,84,120,189,284,384,486,600,726,856,990,
                 1112,1203,1254,1277,1291,1309,1326,1348,1372,1399,1426,1454,1485,1521,1573,1642,1723,
                 1813,1914,2027,2164,2332,2542,2795,3068,3366,3716,4116,4548,4989,5435,5901,6401,6942,
                 7527,8140,8771,9406,10051,10736,11478,12273,13128,14057,15076,16204,17483,18923,20533,
                 22439,24653,27135,29737,32404,35321,38796,42852,47377,52749,58353,64396,71496,79699,88491,
                 97689,109336,122073,135933,150930,167061,184300,202596,221873,242028,262931,284431,306354,
                 328515,350719])/1000000 
GDR

array([5.49800e-03, 4.19000e-04, 2.83000e-04, 2.10000e-04, 1.83000e-04,
       1.51000e-04, 1.31000e-04, 1.14000e-04, 9.80000e-05, 8.30000e-05,
       7.50000e-05, 8.40000e-05, 1.20000e-04, 1.89000e-04, 2.84000e-04,
       3.84000e-04, 4.86000e-04, 6.00000e-04, 7.26000e-04, 8.56000e-04,
       9.90000e-04, 1.11200e-03, 1.20300e-03, 1.25400e-03, 1.27700e-03,
       1.29100e-03, 1.30900e-03, 1.32600e-03, 1.34800e-03, 1.37200e-03,
       1.39900e-03, 1.42600e-03, 1.45400e-03, 1.48500e-03, 1.52100e-03,
       1.57300e-03, 1.64200e-03, 1.72300e-03, 1.81300e-03, 1.91400e-03,
       2.02700e-03, 2.16400e-03, 2.33200e-03, 2.54200e-03, 2.79500e-03,
       3.06800e-03, 3.36600e-03, 3.71600e-03, 4.11600e-03, 4.54800e-03,
       4.98900e-03, 5.43500e-03, 5.90100e-03, 6.40100e-03, 6.94200e-03,
       7.52700e-03, 8.14000e-03, 8.77100e-03, 9.40600e-03, 1.00510e-02,
       1.07360e-02, 1.14780e-02, 1.22730e-02, 1.31280e-02, 1.40570e-02,
       1.50760e-02, 1.62040e-02, 1.74830e-02, 1.89230e-02, 2.053

In [ ]:
Death_other_causes = np.zeros(T)
Death_other_causes = GDR[50:76]  ### Need only age 50-76
Death_other_causes

array([0.004989, 0.005435, 0.005901, 0.006401, 0.006942, 0.007527,
       0.00814 , 0.008771, 0.009406, 0.010051, 0.010736, 0.011478,
       0.012273, 0.013128, 0.014057, 0.015076, 0.016204, 0.017483,
       0.018923, 0.020533, 0.022439, 0.024653, 0.027135, 0.029737,
       0.032404, 0.035321])

In [ ]:
P_W = np.zeros((n_states,n_states,T))
P_W.shape

(91, 91, 25)

In [ ]:
for t in range(T):
    for s in range(n_states-1):
        P_W[s,Death_state,t] = Death_other_causes[t]
    
    P_W[Death_state,Death_state,t] = 1   ## State D is absorbing

In [ ]:
### Cance States
for t in range(T):
    P_W[CRC_states[0],Death_state,t] = P_W[CRC_states[0],Death_state,t] + Death_from_early
    P_W[CRC_states[1],Death_state,t] = P_W[CRC_states[1],Death_state,t] + Death_from_regional
    P_W[CRC_states[2],Death_state,t] = P_W[CRC_states[2],Death_state,t] + Death_from_advanced
    
    
    ## eraly CRC state
    P_W[CRC_states[0],CRC_states[1],t] = Early_to_regional
    P_W[CRC_states[0],CRC_states[0],t] = 1 - P_W[CRC_states[0],Death_state,t] - P_W[CRC_states[0],CRC_states[1],t]
    

    ## Regional CRC state
    P_W[CRC_states[1],CRC_states[2],t] = Regional_to_advanced
    P_W[CRC_states[1],CRC_states[1],t] = 1 - P_W[CRC_states[1],Death_state,t] - P_W[CRC_states[1],CRC_states[2],t]
    

    ## Advanced CRC state
    P_W[CRC_states[2],CRC_states[2],t] = 1 - P_W[CRC_states[2],Death_state,t] 
    


In [ ]:
### treatment state
for t in range(T):
    P_W[Tr_state[0],Tr_state[0],t] = 1 - P_W[Tr_state[0],Death_state,t] ## State Tr is absorbing
    P_W[Tr_state[1],Tr_state[1],t] = 1 - P_W[Tr_state[1],Death_state,t] ## State Tr is absorbing
    P_W[Tr_state[2],Tr_state[2],t] = 1 - P_W[Tr_state[2],Death_state,t] ## State Tr is absorbing


In [ ]:
##large polyps to cancer
for t in range(T):
    for s in range(1,84):

        position = val_list.index(s)
        num_of_large_polyps = key_list[position][2]
        if (num_of_large_polyps > 0):
            #print(s,num_of_large_polyps)
            P_W[s, CRC_states[0],t] =  1 - (1-Large_adenoma_to_early) ** num_of_large_polyps
        #    print(num_of_large_polyps, P_W[s, CRC_states[0],t])
    #P_W[84, 85] =   6 * Large_adenoma_to_early   ### for 7+ state

In [ ]:

for t in range(T):   ### t: time step
    for s in range(84): ### s: state
        #sum_from_P = np.sum(P[s,0:84])
        sum_for_non_polyp_states = np.sum(P_W[s,84:n_states,t])
        new_probs_for_polyp_states_and_7plus = (P[s,0:84] /sum(P[s,0:84])) * (1 - sum_for_non_polyp_states)
        P_W[s,0:84,t] = new_probs_for_polyp_states_and_7plus
print(P_W[82,0,:])       

[0.76028443 0.75984337 0.75938252 0.75888806 0.75835304 0.75777452
 0.7571683  0.75654428 0.75591631 0.75527845 0.75460103 0.75386724
 0.75308104 0.7522355  0.75131678 0.75030906 0.74919354 0.74792869
 0.74650463 0.74491244 0.74302753 0.74083804 0.7383835  0.7358103
 0.73317281]


In [ ]:
### checking if the sum over cols for all row equals 1
for t in range(T):
    for s in range(P_W.shape[0]):
        if (np.round(np.sum(P_W[s,:,t]),2) != 1):
            print(t,s,np.round(np.sum(P_W[s,:,t]),2))

In [ ]:
##ref: http://www3.mdanderson.org/app/medcalc/index.cfm?pagename=coloncancer
one_year_death_rate_CRC_Tr = np.zeros((3,3)) ### CRC_stage * age-group
one_year_death_rate_CRC_Tr[:,0] = [0.01,0.03,0.6] ## age 50_59
one_year_death_rate_CRC_Tr[:,1] = [0.01, 0.04, 0.44] ## age 60_69
one_year_death_rate_CRC_Tr[:,2] = [0.02, 0.04, 0.49] ## 70+

In [ ]:
one_year_death_rate_CRC_Tr

array([[0.01, 0.01, 0.02],
       [0.03, 0.04, 0.04],
       [0.6 , 0.44, 0.49]])

In [ ]:
def is_obs_possible(st_number,obs_number):
    
    if (obs_number == 0):
        return(True)
    elif (st_number < 84 and obs_number >= 84):
        return(False)
    elif (st_number >= 84 and obs_number < 84):
        return(True)
    elif (st_number >= 84 and obs_number >= 84):
        if (s==0):
            return(True)
        else:
            return(False)
            
    else:
        if (st_number == 0):
            st_num_small_polyps = 0
            st_num_med_polyps = 0
            st_num_large_polyps = 0
        else:
            st_pos = val_list.index(st_number)
            st_num_small_polyps = key_list[st_pos][0]
            st_num_med_polyps = key_list[st_pos][1]
            st_num_large_polyps = key_list[st_pos][2]

        obs_pos = val_list.index(obs_number)
        obs_num_small_polyps = key_list[obs_pos][0]
        obs_num_med_polyps = key_list[obs_pos][1]
        obs_num_large_polyps = key_list[obs_pos][2]

        post_col_small = st_num_small_polyps - obs_num_small_polyps
        post_col_med = st_num_med_polyps - obs_num_med_polyps
        post_col_large = st_num_large_polyps - obs_num_large_polyps

        if (post_col_small < 0 or post_col_med < 0 or post_col_large < 0 ):
            return(False)
        else:
            return(True)

    
    

In [ ]:
def post_col_state_calculator(st_number,obs_number):
    
    if (obs_number == 0):
        return(st_number)
    else:
        if (st_number == 0):
            st_num_small_polyps = 0
            st_num_med_polyps = 0
            st_num_large_polyps = 0
        else:
            st_pos = val_list.index(st_number)
            st_num_small_polyps = key_list[st_pos][0]
            st_num_med_polyps = key_list[st_pos][1]
            st_num_large_polyps = key_list[st_pos][2]

        obs_pos = val_list.index(obs_number)
        obs_num_small_polyps = key_list[obs_pos][0]
        obs_num_med_polyps = key_list[obs_pos][1]
        obs_num_large_polyps = key_list[obs_pos][2]

        post_col_small = st_num_small_polyps - obs_num_small_polyps
        post_col_med = st_num_med_polyps - obs_num_med_polyps
        post_col_large = st_num_large_polyps - obs_num_large_polyps

        if (post_col_small >= 0 and post_col_med >= 0 and post_col_large >= 0 ):
            
            post_col_st = (post_col_small,post_col_med,post_col_large)
            if (post_col_st == (0,0,0)):
                post_col_st = 0
            return(states[post_col_st])
        #else:
          #  print("not possible")

    
    

# TPM for action C (colonoscopy)

In [ ]:
#print(P_W[0,0:84,0])

P_C = np.zeros((n_states,n_states,n_states-2,T)) ### S * S * O * T

In [ ]:
# Done in C++ PreCancer
for t in range(T):
    for o in range(84):
        for s_prime in partial_obs_st:
            for s in range(84):

                post_col_st= post_col_state_calculator(s,o)
                if isinstance(post_col_st, int):
                    P_C[s,s_prime,o,t] = P_W[post_col_st,s_prime,t]
                    #print(s,s_prime,o, P_C[s,s_prime,o])

In [ ]:
# Done in C++ in cancer
for t in range(T):
    for o in range(84):
        for s_prime in CRC_states:
            for s in CRC_states:
                P_C[s,s_prime,o,t] = P_W[s,s_prime,t]
                #print(s,s_prime,o, P_C[s,s_prime,o])

In [ ]:
# Done in C++ after state PCO and above in the cancer state
for t in range(T):
    for o in range(84):  ### missing the CRC
        for s_prime in [Death_state]:
            for s in partial_obs_st:
                P_C[s,s_prime,o,t] = P_W[s,s_prime,t]
               # print(s,s_prime,o, P_C[s,s_prime,o])

In [ ]:
# Done in C++
for t in range(T):
    for o in partial_obs_st:
    
        P_C[Tr_state[0],Death_state,o,t] = Death_other_causes[t] + one_year_death_rate_CRC_Tr[0,int(t/10)]
        P_C[Tr_state[1],Death_state,o,t] = Death_other_causes[t] + one_year_death_rate_CRC_Tr[1,int(t/10)]
        P_C[Tr_state[2],Death_state,o,t] = Death_other_causes[t] + one_year_death_rate_CRC_Tr[2,int(t/10)]
        #print(t,P_C[Tr_state[2],Death_state,t])
        P_C[Death_state,Death_state,o,t] = 1
    

In [ ]:
P_C[84,:,83,0], P_C[84,:,84,0], P_C[87,:,0,0], P_W[1,:,0]

(array([0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.660811, 0.28    , 0.      , 0.      , 

# TPM when observation is in CRC states

In [ ]:
# Done in C++
for t in range(T):
    P_C[CRC_states[0],Tr_state[0],CRC_states[0],t] = 1
    P_C[CRC_states[1],Tr_state[1],CRC_states[1],t] = 1
    P_C[CRC_states[2],Tr_state[2],CRC_states[2],t] = 1
    

In [ ]:
# Done in C++
for t in range(T):
    for o in partial_obs_st:
        P_C[Tr_state[0],Tr_state[0],o,t] = 1 - P_C[Tr_state[0],Death_state,o,t] 
        P_C[Tr_state[1],Tr_state[1],o,t] = 1 - P_C[Tr_state[1],Death_state,o,t] 
        P_C[Tr_state[2],Tr_state[2],o,t] = 1 - P_C[Tr_state[2],Death_state,o,t] 

In [ ]:
cnt = 0
for t in range(T):
    for s in range(n_states):
        for o in partial_obs_st:
            if (is_obs_possible (s,o)):
                #print(s,o)
                if (np.round(sum(P_C[s,:,o,t]),2) != 1):
                    #print(t,s,o,"-----",np.round(np.sum(P_C[s,:,o,t]),3))
                    cnt = cnt  + 1

cnt

0

# Immediate Rewards

In [ ]:
Col_disutility = 11/365 ## one month  ~ 0.0301 // Ref Erlney
r = np.zeros((n_states,n_action,T))
non_CRC_utility= [0.78, 0.78, 0.78, 0.78, 0.78, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.77, 0.75, 0.75,
           0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.73] ### len is T

# Ref Marjan :D immidate reward Quality life of people in different stages of cancer
early_CRC_r = 0.6
reg_CRC_r = 0.5
dis_CRC_r = 0.2
len(non_CRC_utility)

26

In [ ]:
# Lump sum reward (Last expression R_t)  

CRC_states_reward = pd.read_csv("MD Anderson.csv")
print(CRC_states_reward)
#termina_reward[50:75,:]

          Loc        Reg       Dis    Average  Unnamed: 4       Loc1  \
0   64.756437  50.408864  5.323284  40.162861         NaN  64.956437   
1   63.950586  49.812126  5.254497  39.672403         NaN  64.150586   
2   63.098278  49.194237  5.234417  39.175644         NaN  63.298278   
3   62.305072  48.615658  5.233728  38.718153         NaN  62.505072   
4   61.491424  47.996754  5.214224  38.234134         NaN  61.691424   
..        ...        ...       ...        ...         ...        ...   
95   0.557915   0.452056  0.008693   0.339555         NaN   0.757915   
96   0.555553   0.404375  0.008087   0.322672         NaN   0.755553   
97   0.519265   0.361891  0.007539   0.296232         NaN   0.719265   
98   0.483216   0.339112  0.007065   0.276464         NaN   0.683216   
99   0.274975   0.313257  0.006665   0.198299         NaN   0.474975   

         Reg1      Dis1   Average1  
0   50.608863  5.523284  40.362861  
1   50.012126  5.454497  39.872403  
2   49.394237  5.434417 

for t in range(T):
        for s in range(84):
            r[s,0,t] = non_CRC_utility[t]
            r[s,1,t] = r[s,0,t] - Col_disutility

        r[CRC_states[0],0,t] = early_CRC_r
        r[CRC_states[1],0,t] = reg_CRC_r
        r[CRC_states[2],0,t] = dis_CRC_r

        r[CRC_states[0],1,t] = early_CRC_r
        r[CRC_states[1],1,t] = reg_CRC_r
        r[CRC_states[2],1,t] = dis_CRC_r

        r[Tr_state[0],1,t] = CRC_states_reward["Loc"][t+50]
        r[Tr_state[1],1,t] = CRC_states_reward["Reg"][t+50]
        r[Tr_state[2],1,t] = CRC_states_reward["Dis"][t+50]


In [ ]:
# Cal r(s,W) plus disutil of col for r(s,Co)

med_polyp_disutility = 0.005
large_polyp_disutility = 0.01
num_of_med_polyps = 0
num_of_large_polyps = 0
# PreCancer
for t in range(T):
        for s in range(84):
            r[s,0,t] = non_CRC_utility[t]
            if (s>0):
                position = val_list.index(s)
                num_of_med_polyps = key_list[position][1]
                num_of_large_polyps = key_list[position][2]
                #print(s)
           
                if (num_of_med_polyps > 0):
                    r[s,0,t] = r[s,0,t]  - num_of_med_polyps * med_polyp_disutility
                if(num_of_large_polyps > 0):
                    r[s,0,t] = r[s,0,t]  - num_of_large_polyps * large_polyp_disutility
                
            
            r[s,1,t] = non_CRC_utility[t] - Col_disutility
# Cancer States
        r[CRC_states[0],0,t] = early_CRC_r
        r[CRC_states[1],0,t] = reg_CRC_r
        r[CRC_states[2],0,t] = dis_CRC_r

        r[CRC_states[0],1,t] = early_CRC_r
        r[CRC_states[1],1,t] = reg_CRC_r
        r[CRC_states[2],1,t] = dis_CRC_r

        r[Tr_state[0],1,t] = CRC_states_reward["Loc"][t+50]
        r[Tr_state[1],1,t] = CRC_states_reward["Reg"][t+50]
        r[Tr_state[2],1,t] = CRC_states_reward["Dis"][t+50]


In [ ]:
r.shape

(91, 2, 25)

# Terminal rewards -- Last period T

In [ ]:
#ref: https://www.ssa.gov/oact/STATS/table4c6.html
terminal_reward = np.zeros(n_states)
for s in range(84):
    terminal_reward[s] = 11.4
    
terminal_reward[CRC_states[0]] = r[Tr_state[0],1,T-1] - 2   #### reward wo treatment
terminal_reward[CRC_states[1]] = r[Tr_state[1],1,T-1] - 1
terminal_reward[CRC_states[2]] = r[Tr_state[2],1,T-1] - 0.2

    
terminal_reward[Tr_state[0]] = r[Tr_state[0],1,T-1] 
terminal_reward[Tr_state[1]] = r[Tr_state[1],1,T-1]
terminal_reward[Tr_state[2]] = r[Tr_state[2],1,T-1] 

terminal_reward

array([11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4 

In [ ]:
### We can use TPM to estimate terminal reward for CRC states.
# Search if P_W20 ever used
from numpy.linalg import matrix_power
P_W20 = matrix_power(P_W[:,:,0], 20)
np.sum(P_W20[0,:]), P_W20[0,:]

(1.0000000000000082,
 array([5.28022725e-01, 2.08649286e-01, 3.73139926e-02, 1.00116926e-02,
        5.59388588e-03, 1.89938240e-02, 3.52225289e-03, 2.52928162e-03,
        1.96886814e-03, 1.38786812e-03, 8.04354131e-04, 1.23978292e-02,
        1.62146453e-03, 7.62742768e-04, 1.17523701e-03, 8.64767693e-04,
        1.71068722e-04, 1.19821120e-04, 5.80838257e-04, 1.48381984e-04,
        2.21302817e-03, 7.48204941e-03, 3.30325621e-03, 8.88615986e-04,
        2.15130663e-04, 1.27147596e-03, 7.28466479e-04, 2.23450004e-04,
        6.50008234e-05, 2.00045948e-04, 1.07503187e-04, 1.02112450e-04,
        8.11140878e-05, 7.68948431e-05, 1.53621418e-05, 5.11583680e-04,
        4.35211861e-03, 7.29713390e-04, 1.34112148e-04, 5.45074908e-04,
        2.61472464e-04, 5.93222270e-04, 8.46186513e-04, 1.74308095e-04,
        2.11898151e-05, 2.71218533e-05, 2.13510849e-05, 6.22749046e-04,
        7.25094706e-05, 3.98709673e-06, 1.27085960e-04, 3.37050649e-04,
        1.52118867e-04, 8.64145658e-06, 1.1

# Q: Info Matrix

In [ ]:
Q1 = pd.read_csv("infor_mat.csv")

In [ ]:
Q2 = Q1.values

NameError: ignored

In [ ]:
Q2.shape

In [ ]:
Q = np.delete(Q2,0,1)

In [ ]:
Q = np.delete(Q,0,0)

In [ ]:
Q.shape

In [ ]:
### removing 7+ state
Q = np.delete(Q,84,1)

In [ ]:
### removing 7+ state
Q = np.delete(Q,84,0)

In [ ]:
Q.shape

In [ ]:
ro = np.zeros(85)
Q = np.insert(Q,85,ro,axis= 0)
Q = np.insert(Q,86,ro,axis= 0)
Q = np.insert(Q,87,ro,axis= 0)
Q = np.insert(Q,88,ro,axis= 0)
Q = np.insert(Q,89,ro,axis= 0)
Q = np.insert(Q,90,ro,axis= 0)

In [ ]:
c = np.zeros(91)
Q = np.insert(Q,85,c,axis= 1)
Q = np.insert(Q,86,c,axis= 1)
Q = np.insert(Q,87,c,axis= 1)
Q = np.insert(Q,88,c,axis= 1)
Q = np.insert(Q,89,c,axis= 1)
Q = np.insert(Q,90,c,axis= 1)

In [ ]:
Q.shape

In [ ]:
CRC_sensitivity = 0.95
Q[84,84] = CRC_sensitivity
Q[85,85] = CRC_sensitivity
Q[86,86] = CRC_sensitivity


In [ ]:
# If we are in cancer and missed it, it can be any of pre cancer states, for now! Think about it guys!

other_obs_prob = np.repeat((1-CRC_sensitivity)/84, 84)
other_obs_prob

In [ ]:
Q[CRC_states[0],0:84] = other_obs_prob
Q[CRC_states[1],0:84] = other_obs_prob
Q[CRC_states[2],0:84] = other_obs_prob

In [ ]:
CRC_states

In [ ]:
for s in Tr_state:
    Q[s,s] = 1
Q[Death_state,Death_state] = 1

In [ ]:
Q = Q.astype(np.float)

In [ ]:
for s in range(n_states):
    Q[s,:] = Q[s,:] / np.sum(Q[s,:] )
    print(s,round(np.sum(Q[s,:]),3))
    

In [ ]:
def return_state_num (n_small, n_med, n_large):
    if (n_small == 0 and n_med == 0 and n_large == 0):
        st_num = 0
    else:
        st_str = (n_small,n_med,n_large)
        st_num =  states.get(st_str)
    return(st_num)
    
    

# Updating Beilef states

In [ ]:
def bayes_update(b,a,my_o,t):
    
    
    if (a == 0):     
        updated_b = np.dot(b,P_W[:,:,t])#update
        #updated_b = updated_b/np.sum(updated_b)
        #print("action", a, updated_b)

    elif (a ==1):
       
        #my_o = np.int(np.random.choice(np.asarray(states) , 1, p = Q_1[my_s,:]))
        if (my_o == 84 or my_o == 85 or my_o == 86):
            #print(my_o, "CRC state")
            updated_b = np.zeros(n_states)
            updated_b[my_o] = 1
            #print(updated_b, np.sum(updated_b))
        #elif (my_o == Tr_state or my_o == Death_state ):
         #   updated_b = np.zeros(n_states)
         #   updated_b[my_o] = 1
        else:
            #print('else')
            updated_b = np.dot(b * Q[:,my_o] , P_C[:,:,my_o,t]) # * Q_U 
            #updated_b = updated_b/np.sum(updated_b)
    for s in Tr_state:
        updated_b [s] = 0
    updated_b [Death_state] = 0
    
    updated_b = updated_b / np.sum(updated_b)
    
    return(updated_b)


In [ ]:
def give_max_Q_PWLC (b_prime, Q_val_PWLC):
    vec= np.zeros((len(actions), n_vectors))
    for a in actions:
        for elem in range(n_vectors):
            vec[a, elem] = b_prime.dot(Q_val_PWLC[:,a,elem])
    #print(vec)
    #max_a, max_elem = np.array(np.where(vec == vec.max())).flatten()
    max_a, max_elem = np.unravel_index(np.argmax(vec, axis=None), vec.shape)
    #print(a,vec,max_a,max_elem,vec[max_a,max_elem])
    return(max_a,max_elem,vec[max_a,max_elem])

In [ ]:
def derive_policy (belief):
    t=0
    qalys = 0
    terminal_qalys = 0
    c0 = 0
    c1 = 0
    action_traj = []
    obs_traj = []
    crc_risk = []
    while(t <= T):
        
        if (t == T):
            terminal_qalys =  np.dot(belief,terminal_reward)
        else:
            res = give_max_Q_PWLC(belief,Q_val_PWLC)
            a = res[0]
            crc_risk.append(np.sum(belief[84:88]))
            qalys = qalys + np.dot(belief,r[:,a,t])
            action_traj.append(a)
            s = choices(state_list, belief)[0]
            #print('------s-----',s,key_list[s])

            #if (s in Tr_state  or s == Death_state):
            #    my_o = s
            if (a==0):
                my_o = 0
                obs_traj.append(-1)
            else:
                my_Q = Q
                #my_s = copy.deepcopy(s) #choices(states,b)[0]
                my_o = np.int(np.random.choice(np.asarray(obs_list) , 1, p = my_Q[s,0:len(obs_list)]))
                obs_traj.append(key_list[my_o])
            #print('my_o',key_list[my_o])
            
            
            

            if (a == 1):
                c1 += 1
                if(my_o in range(84)):
                    belief = bayes_update(belief,a,my_o,t)
                    
                    #print(t,"co","obs", key_list[my_o],"belief",belief,"post_col_st",key_list[s],qalys)
                else:
                    #print(a,"model terminated")
                    break
            
            if (a == 0):
                c0 += 1
                belief = bayes_update(belief,a,my_o,t)
            

                #print(t,"wait",belief,qalys)
                
                
                    

                    #s = post_col_state_calculator(s,my_o)
                    

            
            

        t = t + 1
        #print(t, qalys,terminal_qalys)
    return(action_traj,obs_traj,crc_risk)
    

In [ ]:
def time_with_co_action(lst, item):
    return [i for i, x in enumerate(lst) if x != item]

# estimating initial beilef state

In [ ]:
st_freq_raw = pd.read_csv("st_freq_all_data.csv")
st_freq = list(st_freq_raw['x'])
#st_freq = st_freq / np.sum(st_freq)
num_CRC = st_freq[84]
st_freq = st_freq[0:84]
#st_freq
len(st_freq)

84

In [ ]:
## ref: https://www.cancer.org/content/dam/cancer-org/research/cancer-facts-and-statistics/colorectal-cancer-facts-and-figures/colorectal-cancer-facts-and-figures-2020-2022.pdf
CRC_stage_dist = [37,35,21]
CRC_stage_dist = CRC_stage_dist/np.sum(CRC_stage_dist)
CRC_stage_dist

st_freq.append(num_CRC * CRC_stage_dist[0] ) 
st_freq.append(num_CRC * CRC_stage_dist[1] ) 
st_freq.append(num_CRC * CRC_stage_dist[2] ) 


st_freq


[7828,
 762,
 152,
 121,
 405,
 70,
 45,
 50,
 20,
 41,
 290,
 35,
 16,
 26,
 25,
 7,
 3,
 16,
 3,
 29,
 149,
 36,
 13,
 7,
 21,
 20,
 6,
 1,
 5,
 5,
 4,
 4,
 4,
 0,
 16,
 98,
 14,
 4,
 7,
 4,
 9,
 5,
 5,
 1,
 0,
 2,
 6,
 2,
 0,
 1,
 4,
 3,
 0,
 6,
 2,
 9,
 64,
 12,
 9,
 3,
 3,
 4,
 5,
 6,
 4,
 1,
 1,
 0,
 1,
 3,
 3,
 1,
 1,
 2,
 4,
 0,
 0,
 1,
 2,
 1,
 1,
 3,
 0,
 2,
 79.17204301075269,
 74.89247311827957,
 44.935483870967744]

In [ ]:
zero_val_indices_in_st_freq = np.where (np.array(st_freq) == 0)

In [ ]:
type(list(zero_val_indices_in_st_freq))

list

In [ ]:
st_freq[33]

0

In [ ]:
for i in range(len(zero_val_indices_in_st_freq[0])):
    #print(i)
    print(zero_val_indices_in_st_freq[0][i])
    st_freq[zero_val_indices_in_st_freq[0][i]] = 1


33
44
48
52
67
75
76
82


In [ ]:
st_freq.append (0)
st_freq.append (0)
st_freq.append (0)
st_freq.append (0)

In [ ]:
st_dist = st_freq /np.sum(st_freq)
st_dist

st_dist

array([7.27441688e-01, 7.08112629e-02, 1.41250813e-02, 1.12443081e-02,
       3.76359074e-02, 6.50497166e-03, 4.18176749e-03, 4.64640833e-03,
       1.85856333e-03, 3.81005483e-03, 2.69491683e-02, 3.25248583e-03,
       1.48685066e-03, 2.41613233e-03, 2.32320416e-03, 6.50497166e-04,
       2.78784500e-04, 1.48685066e-03, 2.78784500e-04, 2.69491683e-03,
       1.38462968e-02, 3.34541399e-03, 1.20806616e-03, 6.50497166e-04,
       1.95149150e-03, 1.85856333e-03, 5.57568999e-04, 9.29281665e-05,
       4.64640833e-04, 4.64640833e-04, 3.71712666e-04, 3.71712666e-04,
       3.71712666e-04, 9.29281665e-05, 1.48685066e-03, 9.10696032e-03,
       1.30099433e-03, 3.71712666e-04, 6.50497166e-04, 3.71712666e-04,
       8.36353499e-04, 4.64640833e-04, 4.64640833e-04, 9.29281665e-05,
       9.29281665e-05, 1.85856333e-04, 5.57568999e-04, 1.85856333e-04,
       9.29281665e-05, 9.29281665e-05, 3.71712666e-04, 2.78784500e-04,
       9.29281665e-05, 5.57568999e-04, 1.85856333e-04, 8.36353499e-04,
      

In [ ]:
initial_b = st_dist
print(np.where (initial_b == 0))
initial_b

(array([87, 88, 89, 90]),)


array([7.27441688e-01, 7.08112629e-02, 1.41250813e-02, 1.12443081e-02,
       3.76359074e-02, 6.50497166e-03, 4.18176749e-03, 4.64640833e-03,
       1.85856333e-03, 3.81005483e-03, 2.69491683e-02, 3.25248583e-03,
       1.48685066e-03, 2.41613233e-03, 2.32320416e-03, 6.50497166e-04,
       2.78784500e-04, 1.48685066e-03, 2.78784500e-04, 2.69491683e-03,
       1.38462968e-02, 3.34541399e-03, 1.20806616e-03, 6.50497166e-04,
       1.95149150e-03, 1.85856333e-03, 5.57568999e-04, 9.29281665e-05,
       4.64640833e-04, 4.64640833e-04, 3.71712666e-04, 3.71712666e-04,
       3.71712666e-04, 9.29281665e-05, 1.48685066e-03, 9.10696032e-03,
       1.30099433e-03, 3.71712666e-04, 6.50497166e-04, 3.71712666e-04,
       8.36353499e-04, 4.64640833e-04, 4.64640833e-04, 9.29281665e-05,
       9.29281665e-05, 1.85856333e-04, 5.57568999e-04, 1.85856333e-04,
       9.29281665e-05, 9.29281665e-05, 3.71712666e-04, 2.78784500e-04,
       9.29281665e-05, 5.57568999e-04, 1.85856333e-04, 8.36353499e-04,
      

In [ ]:
initial_b[16], initial_b[31]

(0.00027878449958182325, 0.0003717126661090977)

In [ ]:
state_list = list(states.values())
obs_list = state_list[0:(n_states-4)]
discount_factor = 1
#obs_list


In [ ]:
r[84,1,:] = r[87,1,:]
r[85,1,:] = r[88,1,:]
r[86,1,:] = r[89,1,:]
epsilon = 0.01

In [ ]:
2**25

33554432

In [ ]:
terminal_reward

array([11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
       11.4       , 11.4       , 11.4       , 11.4       , 11.4 

In [ ]:
terminal_reward_w = copy.deepcopy(terminal_reward)
terminal_reward_w [Tr_state[0]] = terminal_reward [CRC_states[0]]
terminal_reward_w [Tr_state[1]] = terminal_reward [CRC_states[1]]
terminal_reward_w [Tr_state[2]] = terminal_reward [CRC_states[2]]

In [ ]:
terminal_reward_co = copy.deepcopy(terminal_reward)
terminal_reward_co [CRC_states[0]] = terminal_reward [Tr_state[0]]
terminal_reward_co [CRC_states[1]] = terminal_reward [Tr_state[1]]
terminal_reward_co [CRC_states[2]] = terminal_reward [Tr_state[2]]

In [ ]:
terminal_reward_w, terminal_reward_co

(array([11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4       , 11.4       ,
        11.4       , 11.4       , 11.4       , 11.4

In [ ]:
terminal_reward_per_action = np.zeros((n_states, n_action))
terminal_reward_per_action[:,0] = terminal_reward_w
terminal_reward_per_action[:,1] = terminal_reward_co
#terminal_reward_per_action

In [ ]:
#r[:,0,0], r[:,1,0]


In [ ]:
init_b_list = []
for o in obs_list:
    u_b = bayes_update(initial_b,1,o,0)
    init_b_list.append(u_b)
   # print(o,u_b)

NameError: ignored

In [ ]:
init_b_list[0]

array([5.14198361e-01, 3.81874269e-01, 2.79429345e-02, 5.11894916e-03,
       3.37098078e-03, 1.67760963e-02, 2.04095578e-03, 1.71757500e-03,
       1.18134797e-03, 3.14785035e-04, 8.18645982e-04, 1.31433567e-02,
       8.42882039e-04, 4.49693534e-04, 5.98943944e-04, 2.56702534e-04,
       1.86896236e-04, 2.50853420e-05, 7.13237219e-04, 1.80367264e-05,
       9.38242749e-04, 7.12282435e-03, 1.85505865e-03, 4.98141064e-04,
       2.77090041e-04, 1.40756592e-03, 7.51948241e-04, 1.25168316e-04,
       1.43719731e-06, 1.86685522e-04, 1.00476331e-04, 8.95819772e-05,
       1.69854801e-05, 9.91429286e-05, 9.18322853e-07, 7.62450169e-04,
       4.17402459e-03, 5.67858018e-04, 5.42193238e-05, 7.88738148e-04,
       1.84996237e-04, 2.96057076e-04, 1.86384156e-04, 2.46221925e-04,
       9.12547685e-07, 5.50178412e-07, 2.10247999e-05, 1.40307474e-04,
       7.94387740e-05, 1.27417157e-08, 1.37959681e-05, 7.41019020e-05,
       6.80491336e-05, 1.36530805e-07, 1.86723906e-04, 1.55973147e-05,
      

In [ ]:
n_vectors = 1
#Q_val_PWLC = np.random.rand(len(states),len(actions), n_vectors)
Q_val_PWLC = np.zeros((len(states),len(actions), n_vectors))
#Q_val_PWLC[:,0,0], Q_val_PWLC.shape
#Q_val_PWLC[0,0,0] = 1
#Q_val_PWLC[83,1,0] = 1

Q_val_PWLC[:,0,0] = r[:,0,T-1]
Q_val_PWLC[:,1,0] = r[:,1,T-1]
Q_val_PWLC


array([[[0.75      ],
        [0.71986301]],

       [[0.75      ],
        [0.71986301]],

       [[0.745     ],
        [0.71986301]],

       [[0.74      ],
        [0.71986301]],

       [[0.75      ],
        [0.71986301]],

       [[0.745     ],
        [0.71986301]],

       [[0.74      ],
        [0.71986301]],

       [[0.74      ],
        [0.71986301]],

       [[0.735     ],
        [0.71986301]],

       [[0.73      ],
        [0.71986301]],

       [[0.75      ],
        [0.71986301]],

       [[0.745     ],
        [0.71986301]],

       [[0.74      ],
        [0.71986301]],

       [[0.735     ],
        [0.71986301]],

       [[0.74      ],
        [0.71986301]],

       [[0.735     ],
        [0.71986301]],

       [[0.73      ],
        [0.71986301]],

       [[0.73      ],
        [0.71986301]],

       [[0.725     ],
        [0.71986301]],

       [[0.72      ],
        [0.71986301]],

       [[0.75      ],
        [0.71986301]],

       [[0.745     ],
        [0.7

# PWLC Q-learning

In [ ]:
### PWLC Q-learning

n_run = 200

for rep in range(n_run):
   # if (rep % 10000 == 0):
    #    print("rep: ", rep)
    

    learning_rate = 0.25 / ( 1 + rep)
    
    #if (rep < 1000):
       # learning_rate = 0.2
    #elif(rep >=1000 and rep <=5000):
       # learning_rate = 0.1
    #else:
        #learning_rate = 0.05
    
    if (False):
        b_non_norm = np.random.rand(len(state_list))#[1,0,0,0,0,0]
        #b_non_norm = np.zeros(n_states)
        #b_non_norm [84] = 1
        b_non_norm[(Tr_state[0])] = 0 ### treatment states: fully observable
        b_non_norm[(Tr_state[1])] = 0 ### treatment states: fully observable
        b_non_norm[(Tr_state[2])] = 0 ### treatment states: fully observable
        b_non_norm[Death_state] = 0 ### death states: fully observable
        b = b_non_norm / sum(b_non_norm)
        
    rand_baseline_obs =  choices(obs_list[0:84])[0]
    b = init_b_list[rand_baseline_obs]
    if (rep % 10000 == 0):
        print("rep: ", rep)
        print("b----",rand_baseline_obs,b)
    s = choices(state_list, b)[0]
    #print('s', s)
    for t in range(T+1):
        
        if (t == T ):
            a,elem = give_max_Q_PWLC (b, Q_val_PWLC) [0:2] 
            
            for s_1 in state_list:
                Q_val_PWLC [s_1,a, elem] = Q_val_PWLC [s_1,a, elem] + learning_rate * b[s_1] * (terminal_reward_per_action[s_1,a] - q_a_dot_b)
            
    #        print("Last t", t,a, Q_val_PWLC.transpose())    
        
        else: 
            
            rand_num = random.uniform(0, 1)
            if (rand_num < epsilon):
                a = random.choice(actions)
 #               print("exploration", a)
            else:
                a,elem = give_max_Q_PWLC (b, Q_val_PWLC) [0:2]  #np.argmin([Q_val_PWLC[s,:] for a in actions])
#            print("best action selection: " ,b.dot(Q_val_PWLC[:,0,0]), b.dot(Q_val_PWLC[:,1,0]))
                #print('t,s, b,a,elem', t,s,b,a,elem)

            if (s == Death_state):
 #               print("patient death",Q_val_PWLC.transpose())
                break  ### arbitrary obs...could be any obs
                    
            
            if (a==0):
                my_o = 0
                P = P_W[:,:,t]
            else:
                my_Q = Q
                
                my_o = np.int(np.random.choice(np.asarray(obs_list) , 1, p = my_Q[s,0:len(obs_list)]))
                
                P = P_C[:,:,my_o,t]
                #print("a,b",a,b)

            #print('---my_o', my_o)
            #print('t,s, b,a,elem,my_o', t,key_list[s],b,a,elem,key_list[my_o])

            s_prime = choices(state_list, P[s,:])[0]
            b_prime = bayes_update(b,a,my_o,t)
            max_Q = give_max_Q_PWLC (b_prime, Q_val_PWLC)[2]
            
            #print("---b_prime, arg_min_Q, Q_val [s,a]", np.round(b_prime,2), arg_min_Q, Q_val [s,a])

            #old_Q_val_PWLC = copy.deepcopy(Q_val_PWLC)
            q_a_dot_b  = b.dot(Q_val_PWLC [:,a, elem])

            if (my_o in CRC_states):  ### happens only when a=1 : co
                #print("my_o is in CRC states", my_o)
                for s_1 in CRC_states:   #### only going through CRC states since there is no uncertainiy... e.g., belief would be e_84
                    if (s_1 == CRC_states[0]):
                        rr = r[Tr_state[0],a,t]
                    elif(s_1 == CRC_states[1]):
                        rr = r[Tr_state[1],a,t]
                    elif(s_1 == CRC_states[2]):
                        rr = r[Tr_state[2],a,t]
 #               print("my_o in CRC---", key_list[my_o] )
                s_1 = my_o
                ### if my_o is in CRC we can only be in CRC... no uncertainity left... so we just update the state we observed
                Q_val_PWLC [s_1,a, elem] = Q_val_PWLC [s_1,a, elem] + learning_rate * b[s_1] * (r[s_1,a,t] + rr - q_a_dot_b)
                    #print(s_1,a, my_o, np.round(b,2), Q_val [s,a])
 #               print(rep, learning_rate, t,np.round(b,2), "\n",key_list[s],a,key_list[my_o], "\n",Q_val_PWLC.transpose()  )
            
                break

            #elif (s in Tr_state or s == Death_state or my_o in Tr_state or my_o == Death_state):
            #elif ( s == Death_state):
           #     Q_val_PWLC [s,a, elem] = Q_val_PWLC [s_1,a, elem] + learning_rate * 0
                #break

            else:

                for s_1 in state_list:
                    Q_val_PWLC [s_1,a, elem] = Q_val_PWLC [s_1,a, elem] + learning_rate * b[s_1] * (r[s_1,a,t] + discount_factor * max_Q - q_a_dot_b )
                    #print(s_1,a, my_o, np.round(b,2), Q_val [s,a])

 #           print(rep,learning_rate, t,np.round(b,2), "\n",key_list[s],a,key_list[my_o], "\n",Q_val_PWLC.transpose()  )
            #print("---->>>>>>>>>", abs(np.sum(old_Q_val_PWLC - Q_val_PWLC)))


            #print(s,s_prime )
            s = copy.deepcopy(s_prime) # choices(states,b_prime)[0]
            b = copy.deepcopy(b_prime)
            #print(s,s_prime)
        Q_val_PWLC_old = copy.deepcopy(Q_val_PWLC)
  #      print("---------")
 #   print(">>>>>>>>>>>>>>>>>>> next replication")

rep:  0
b---- 72 [4.16238316e-01 3.23517695e-01 2.14294079e-02 3.71279523e-03
 2.56850106e-03 1.30785096e-02 1.53021259e-03 1.35638507e-03
 9.22648322e-04 1.56779606e-04 6.73250853e-04 1.05396623e-02
 6.31775221e-04 3.56970634e-04 4.34437146e-04 1.76305492e-04
 1.56682602e-04 1.84428216e-05 5.91231560e-04 6.38772250e-06
 6.27809253e-04 5.64606660e-03 1.36740374e-03 3.70726475e-04
 2.33613579e-04 1.14570241e-03 6.17290287e-04 1.01616651e-04
 2.16145091e-07 1.54652858e-04 8.36050810e-05 7.55469811e-05
 7.43366307e-06 8.28970453e-05 4.52553913e-07 6.37059774e-04
 3.31209082e-03 4.46130949e-04 4.18822576e-05 6.61288556e-04
 1.40911411e-04 2.08645093e-04 9.43303352e-05 2.07440856e-04
 2.59532977e-08 1.08129663e-07 1.75660493e-05 7.82673593e-05
 6.67325574e-05 6.74270967e-11 4.49809812e-11 3.61304499e-05
 4.93779239e-05 2.80783109e-12 1.58084807e-04 1.01262466e-05
 5.24975909e-05 3.41475329e-03 3.66883003e-04 4.31683744e-04
 1.48323914e-04 1.07804434e-04 1.33992458e-04 1.61981473e-04
 1.9004

In [ ]:
### results for when we are certin in which state we are

for s in obs_list:
    my_b = np.zeros(n_states)
    my_b [s] = 1
    res = give_max_Q_PWLC(my_b,Q_val_PWLC)
    a = res[0]
    print(s,key_list[s],a)

0 0 1
1 (1, 0, 0) 1
2 (0, 1, 0) 1
3 (0, 0, 1) 1
4 (2, 0, 0) 1
5 (1, 1, 0) 1
6 (0, 2, 0) 1
7 (1, 0, 1) 1
8 (0, 1, 1) 1
9 (0, 0, 2) 1
10 (3, 0, 0) 1
11 (2, 1, 0) 1
12 (1, 2, 0) 1
13 (0, 3, 0) 1
14 (2, 0, 1) 1
15 (1, 1, 1) 1
16 (0, 2, 1) 1
17 (1, 0, 2) 0
18 (0, 1, 2) 1
19 (0, 0, 3) 1
20 (4, 0, 0) 1
21 (3, 1, 0) 1
22 (2, 2, 0) 1
23 (1, 3, 0) 1
24 (0, 4, 0) 1
25 (3, 0, 1) 1
26 (2, 1, 1) 1
27 (1, 2, 1) 0
28 (0, 3, 1) 0
29 (2, 0, 2) 1
30 (1, 1, 2) 1
31 (0, 2, 2) 1
32 (1, 0, 3) 1
33 (0, 1, 3) 1
34 (0, 0, 4) 1
35 (5, 0, 0) 1
36 (4, 1, 0) 1
37 (3, 2, 0) 1
38 (2, 3, 0) 0
39 (1, 4, 0) 1
40 (0, 5, 0) 1
41 (4, 0, 1) 1
42 (3, 1, 1) 1
43 (2, 2, 1) 1
44 (1, 3, 1) 0
45 (0, 4, 1) 1
46 (3, 0, 2) 0
47 (2, 1, 2) 1
48 (1, 2, 2) 1
49 (0, 3, 2) 1
50 (2, 0, 3) 1
51 (1, 1, 3) 1
52 (0, 2, 3) 1
53 (1, 0, 4) 1
54 (0, 1, 4) 1
55 (0, 0, 5) 1
56 (6, 0, 0) 0
57 (5, 1, 0) 1
58 (4, 2, 0) 1
59 (3, 3, 0) 1
60 (2, 4, 0) 0
61 (1, 5, 0) 1
62 (0, 6, 0) 1
63 (5, 0, 1) 0
64 (4, 1, 1) 1
65 (3, 2, 1) 0
66 (2, 3, 1) 1
67 (1, 4, 1) 

In [ ]:
return_state_num(2,0,0)

4

In [ ]:
rep

199

In [ ]:
Q_val_PWLC

array([[[ 4.84061842],
        [36.17849695]],

       [[ 3.36322517],
        [22.22649798]],

       [[ 1.00691766],
        [ 3.00878456]],

       [[ 0.79779447],
        [ 1.25857193]],

       [[ 0.78189556],
        [ 1.01560393]],

       [[ 0.89154702],
        [ 1.99844536]],

       [[ 0.76004078],
        [ 0.90844391]],

       [[ 0.75525286],
        [ 0.86192174]],

       [[ 0.74509478],
        [ 0.8182039 ]],

       [[ 0.73570036],
        [ 0.77528604]],

       [[ 0.75618757],
        [ 0.77242747]],

       [[ 0.85176932],
        [ 1.63612014]],

       [[ 0.74876951],
        [ 0.80302995]],

       [[ 0.7391699 ],
        [ 0.75872699]],

       [[ 0.74664675],
        [ 0.78160741]],

       [[ 0.73816696],
        [ 0.75061119]],

       [[ 0.73134842],
        [ 0.73172934]],

       [[ 0.73031838],
        [ 0.72304249]],

       [[ 0.73038887],
        [ 0.76592394]],

       [[ 0.7204897 ],
        [ 0.72545007]],

       [[ 0.76160896],
        [ 0.82573

In [ ]:
b_prime

array([6.11687585e-01, 2.40037941e-01, 2.86644546e-02, 1.86208553e-02,
       9.65852232e-03, 1.57238202e-02, 6.38104134e-03, 5.56851079e-03,
       4.51838246e-03, 1.48939131e-03, 6.68642447e-04, 1.09648381e-02,
       3.17651396e-03, 1.65045640e-03, 1.91419530e-03, 7.82809649e-04,
       3.61698396e-04, 1.87237949e-04, 9.27486376e-04, 5.15920779e-04,
       1.45021181e-03, 6.29714219e-03, 4.21773597e-03, 8.88120122e-04,
       1.89432867e-04, 1.57607509e-03, 9.70149325e-04, 1.15539258e-04,
       1.92874231e-04, 4.43632657e-04, 8.26196491e-05, 8.88985336e-05,
       7.77461651e-05, 7.24464871e-05, 1.10309097e-05, 6.14190034e-04,
       3.67811942e-03, 6.21262581e-04, 2.78054693e-04, 7.59923167e-04,
       2.28702427e-04, 4.69866671e-04, 1.01213933e-03, 2.48667032e-04,
       1.01280868e-05, 1.16256475e-04, 2.77559868e-05, 1.72319585e-03,
       8.49956557e-05, 3.94561752e-09, 7.40992801e-05, 2.86052913e-04,
       3.65497557e-04, 2.06541113e-05, 1.18925903e-04, 1.88362864e-04,
      

In [ ]:
Q_val_PWLC

array([[[ 4.84061842],
        [36.17849695]],

       [[ 3.36322517],
        [22.22649798]],

       [[ 1.00691766],
        [ 3.00878456]],

       [[ 0.79779447],
        [ 1.25857193]],

       [[ 0.78189556],
        [ 1.01560393]],

       [[ 0.89154702],
        [ 1.99844536]],

       [[ 0.76004078],
        [ 0.90844391]],

       [[ 0.75525286],
        [ 0.86192174]],

       [[ 0.74509478],
        [ 0.8182039 ]],

       [[ 0.73570036],
        [ 0.77528604]],

       [[ 0.75618757],
        [ 0.77242747]],

       [[ 0.85176932],
        [ 1.63612014]],

       [[ 0.74876951],
        [ 0.80302995]],

       [[ 0.7391699 ],
        [ 0.75872699]],

       [[ 0.74664675],
        [ 0.78160741]],

       [[ 0.73816696],
        [ 0.75061119]],

       [[ 0.73134842],
        [ 0.73172934]],

       [[ 0.73031838],
        [ 0.72304249]],

       [[ 0.73038887],
        [ 0.76592394]],

       [[ 0.7204897 ],
        [ 0.72545007]],

       [[ 0.76160896],
        [ 0.82573

In [ ]:
### results for when we are certin in which state we are

for s in obs_list:
    my_b = np.zeros(n_states)
    my_b [s] = 1
    res = give_max_Q_PWLC(my_b,Q_val_PWLC)
    a = res[0]
    print(s,key_list[s],a)

0 0 1
1 (1, 0, 0) 1
2 (0, 1, 0) 1
3 (0, 0, 1) 1
4 (2, 0, 0) 1
5 (1, 1, 0) 1
6 (0, 2, 0) 1
7 (1, 0, 1) 1
8 (0, 1, 1) 1
9 (0, 0, 2) 1
10 (3, 0, 0) 1
11 (2, 1, 0) 1
12 (1, 2, 0) 1
13 (0, 3, 0) 1
14 (2, 0, 1) 1
15 (1, 1, 1) 1
16 (0, 2, 1) 1
17 (1, 0, 2) 0
18 (0, 1, 2) 1
19 (0, 0, 3) 1
20 (4, 0, 0) 1
21 (3, 1, 0) 1
22 (2, 2, 0) 1
23 (1, 3, 0) 1
24 (0, 4, 0) 1
25 (3, 0, 1) 1
26 (2, 1, 1) 1
27 (1, 2, 1) 0
28 (0, 3, 1) 0
29 (2, 0, 2) 1
30 (1, 1, 2) 1
31 (0, 2, 2) 1
32 (1, 0, 3) 1
33 (0, 1, 3) 1
34 (0, 0, 4) 1
35 (5, 0, 0) 1
36 (4, 1, 0) 1
37 (3, 2, 0) 1
38 (2, 3, 0) 0
39 (1, 4, 0) 1
40 (0, 5, 0) 1
41 (4, 0, 1) 1
42 (3, 1, 1) 1
43 (2, 2, 1) 1
44 (1, 3, 1) 0
45 (0, 4, 1) 1
46 (3, 0, 2) 0
47 (2, 1, 2) 1
48 (1, 2, 2) 1
49 (0, 3, 2) 1
50 (2, 0, 3) 1
51 (1, 1, 3) 1
52 (0, 2, 3) 1
53 (1, 0, 4) 1
54 (0, 1, 4) 1
55 (0, 0, 5) 1
56 (6, 0, 0) 0
57 (5, 1, 0) 1
58 (4, 2, 0) 1
59 (3, 3, 0) 1
60 (2, 4, 0) 0
61 (1, 5, 0) 1
62 (0, 6, 0) 1
63 (5, 0, 1) 0
64 (4, 1, 1) 1
65 (3, 2, 1) 0
66 (2, 3, 1) 1
67 (1, 4, 1) 

In [ ]:
key_list[56]

(6, 0, 0)

In [ ]:
return_state_num(2, 3, 0),Q[38,:]

(38, array([2.10937999e-04, 1.26562499e-03, 3.58593799e-03, 0.00000000e+00,
        1.89843799e-03, 2.15156249e-02, 2.03203129e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.22734379e-02,
        1.21921875e-01, 3.83828128e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.82882812e-01, 2.30296874e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.45445312e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00

In [ ]:
Q[:,86]

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.95, 0.  ,
       0.  , 0.  , 0.  ])

In [ ]:
b

array([6.11687585e-01, 2.40037941e-01, 2.86644546e-02, 1.86208553e-02,
       9.65852232e-03, 1.57238202e-02, 6.38104134e-03, 5.56851079e-03,
       4.51838246e-03, 1.48939131e-03, 6.68642447e-04, 1.09648381e-02,
       3.17651396e-03, 1.65045640e-03, 1.91419530e-03, 7.82809649e-04,
       3.61698396e-04, 1.87237949e-04, 9.27486376e-04, 5.15920779e-04,
       1.45021181e-03, 6.29714219e-03, 4.21773597e-03, 8.88120122e-04,
       1.89432867e-04, 1.57607509e-03, 9.70149325e-04, 1.15539258e-04,
       1.92874231e-04, 4.43632657e-04, 8.26196491e-05, 8.88985336e-05,
       7.77461651e-05, 7.24464871e-05, 1.10309097e-05, 6.14190034e-04,
       3.67811942e-03, 6.21262581e-04, 2.78054693e-04, 7.59923167e-04,
       2.28702427e-04, 4.69866671e-04, 1.01213933e-03, 2.48667032e-04,
       1.01280868e-05, 1.16256475e-04, 2.77559868e-05, 1.72319585e-03,
       8.49956557e-05, 3.94561752e-09, 7.40992801e-05, 2.86052913e-04,
       3.65497557e-04, 2.06541113e-05, 1.18925903e-04, 1.88362864e-04,
      

In [ ]:
bayes_update(b,1,84,0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0.])

In [ ]:
Q_val_PWLC

array([[[ 4.84061842],
        [36.17849695]],

       [[ 3.36322517],
        [22.22649798]],

       [[ 1.00691766],
        [ 3.00878456]],

       [[ 0.79779447],
        [ 1.25857193]],

       [[ 0.78189556],
        [ 1.01560393]],

       [[ 0.89154702],
        [ 1.99844536]],

       [[ 0.76004078],
        [ 0.90844391]],

       [[ 0.75525286],
        [ 0.86192174]],

       [[ 0.74509478],
        [ 0.8182039 ]],

       [[ 0.73570036],
        [ 0.77528604]],

       [[ 0.75618757],
        [ 0.77242747]],

       [[ 0.85176932],
        [ 1.63612014]],

       [[ 0.74876951],
        [ 0.80302995]],

       [[ 0.7391699 ],
        [ 0.75872699]],

       [[ 0.74664675],
        [ 0.78160741]],

       [[ 0.73816696],
        [ 0.75061119]],

       [[ 0.73134842],
        [ 0.73172934]],

       [[ 0.73031838],
        [ 0.72304249]],

       [[ 0.73038887],
        [ 0.76592394]],

       [[ 0.7204897 ],
        [ 0.72545007]],

       [[ 0.76160896],
        [ 0.82573

In [ ]:
my_b = np.zeros(n_states)
my_b [0] = 0
my_b [1] = 0.13
my_b [2] = 0.05
my_b [3] = 0
my_b [9] = 0.7
give_max_Q_PWLC(my_b,Q_val_PWLC)

(1, 0, 3.582584196230852)

In [ ]:
import pickle


with open('Q_val_PWLC_1.pickle', 'wb') as handle:
    pickle.dump(Q_val_PWLC, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
### readin pickle
with open('Q_val_PWLC_1.pickle', 'rb') as handle:
    Q_mat = pickle.load(handle)

In [ ]:
Q_val_PWLC

array([[[ 4.84061842],
        [36.17849695]],

       [[ 3.36322517],
        [22.22649798]],

       [[ 1.00691766],
        [ 3.00878456]],

       [[ 0.79779447],
        [ 1.25857193]],

       [[ 0.78189556],
        [ 1.01560393]],

       [[ 0.89154702],
        [ 1.99844536]],

       [[ 0.76004078],
        [ 0.90844391]],

       [[ 0.75525286],
        [ 0.86192174]],

       [[ 0.74509478],
        [ 0.8182039 ]],

       [[ 0.73570036],
        [ 0.77528604]],

       [[ 0.75618757],
        [ 0.77242747]],

       [[ 0.85176932],
        [ 1.63612014]],

       [[ 0.74876951],
        [ 0.80302995]],

       [[ 0.7391699 ],
        [ 0.75872699]],

       [[ 0.74664675],
        [ 0.78160741]],

       [[ 0.73816696],
        [ 0.75061119]],

       [[ 0.73134842],
        [ 0.73172934]],

       [[ 0.73031838],
        [ 0.72304249]],

       [[ 0.73038887],
        [ 0.76592394]],

       [[ 0.7204897 ],
        [ 0.72545007]],

       [[ 0.76160896],
        [ 0.82573

In [ ]:
sum(init_b_list[6][84:87]), sum(init_b_list[16][84:87]),sum(init_b_list[31][84:87]),sum(init_b_list[52][84:87]),sum(init_b_list[80][84:87])


(0.002144204676775614,
 0.023781527008336738,
 0.02769185157588215,
 0.07666583269182331,
 0.1339033383581807)

In [ ]:
for i in range(len(init_b_list)):
    my_b = init_b_list[i]
    a_list,o_list,r_list = derive_policy(my_b)
    time_with_co = time_with_co_action(o_list,-1)
    print(i,key_list[i], time_with_co, sum(my_b[84:87]), my_b)

0 0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] 5.587660132954738e-05 [5.14198361e-01 3.81874269e-01 2.79429345e-02 5.11894916e-03
 3.37098078e-03 1.67760963e-02 2.04095578e-03 1.71757500e-03
 1.18134797e-03 3.14785035e-04 8.18645982e-04 1.31433567e-02
 8.42882039e-04 4.49693534e-04 5.98943944e-04 2.56702534e-04
 1.86896236e-04 2.50853420e-05 7.13237219e-04 1.80367264e-05
 9.38242749e-04 7.12282435e-03 1.85505865e-03 4.98141064e-04
 2.77090041e-04 1.40756592e-03 7.51948241e-04 1.25168316e-04
 1.43719731e-06 1.86685522e-04 1.00476331e-04 8.95819772e-05
 1.69854801e-05 9.91429286e-05 9.18322853e-07 7.62450169e-04
 4.17402459e-03 5.67858018e-04 5.42193238e-05 7.88738148e-04
 1.84996237e-04 2.96057076e-04 1.86384156e-04 2.46221925e-04
 9.12547685e-07 5.50178412e-07 2.10247999e-05 1.40307474e-04
 7.94387740e-05 1.27417157e-08 1.37959681e-05 7.41019020e-05
 6.80491336e-05 1.36530805e-07 1.86723906e-04 1.55973147e-05
 6.37888279e-05 4.11102536e-0

In [ ]:
train_set = []
n_run = 100

b_non_norm = np.random.rand(len(state_list))#[1,0,0,0,0,0]
#b_non_norm = np.zeros(n_states)
#b_non_norm [84] = 1
b_non_norm[(Tr_state[0])] = 0 ### treatment states: fully observable
b_non_norm[(Tr_state[1])] = 0 ### treatment states: fully observable
b_non_norm[(Tr_state[2])] = 0 ### treatment states: fully observable
b_non_norm[Death_state] = 0 ### death states: fully observablemy_b = np.zeros(n_states)
my_b = b_non_norm / np.sum(b_non_norm)
print("my_b", my_b)
    
for run in range(n_run):

    
    res = derive_policy (my_b)
    o_list = res[1]
    risk_list = res[2]
    
    time_with_co = time_with_co_action(o_list,-1)
    
    t_obs_nextCoTime = []
    for i in range(len(time_with_co)-1):
        x = (time_with_co[i],o_list[time_with_co[i]],time_with_co[i+1] - time_with_co[i],risk_list[i])
        t_obs_nextCoTime.append(x)

    train_set.append(t_obs_nextCoTime)

my_b [0.00078598 0.01092319 0.01409919 0.00517506 0.02142162 0.01572316
 0.01829417 0.01649801 0.01170404 0.0145487  0.00922902 0.01167277
 0.02224918 0.02024668 0.00754736 0.01743891 0.02249838 0.01256625
 0.01997053 0.01373068 0.00286223 0.00152548 0.00110837 0.00190223
 0.00114609 0.00221643 0.01534413 0.00013515 0.00302714 0.00611513
 0.01719948 0.00273939 0.0196941  0.01214319 0.01984362 0.00476139
 0.00207702 0.00600492 0.01519161 0.00576392 0.01892146 0.02142173
 0.0050786  0.00514041 0.01412431 0.00988361 0.00728249 0.02118939
 0.01518075 0.01122968 0.01478429 0.01757811 0.0108058  0.02305824
 0.02157765 0.00186979 0.01835964 0.00103282 0.00024781 0.02057943
 0.01710124 0.00095928 0.01269764 0.00208827 0.01862929 0.01464519
 0.01306889 0.00061366 0.00882313 0.01905731 0.00114161 0.00647005
 0.01902248 0.01259958 0.00087759 0.01465376 0.00679178 0.00405311
 0.01224037 0.01728355 0.01428417 0.01737516 0.01290001 0.01596262
 0.01208322 0.02160752 0.0164996  0.         0.         0

In [ ]:
train_set

[[],
 [(0, (0, 2, 0), 1, 0.05019034821910376),
  (1, (1, 0, 0), 1, 0.002828706934349517),
  (2, 0, 1, 7.698025551826231e-05),
  (3, 0, 1, 2.7928160835247956e-05),
  (4, 0, 1, 3.373836077654783e-05),
  (5, (0, 1, 0), 1, 3.336820369461663e-05),
  (6, (1, 0, 0), 1, 8.754522010846677e-05),
  (7, 0, 1, 2.2076349007024412e-05),
  (8, (0, 1, 0), 1, 2.5061688861170278e-05),
  (9, (1, 0, 0), 1, 8.332721326961849e-05),
  (10, 0, 1, 2.196488516088758e-05),
  (11, (1, 0, 0), 1, 2.5092025316872682e-05),
  (12, 0, 1, 2.157605055485056e-05),
  (13, 0, 1, 2.526733631554991e-05),
  (14, (0, 1, 0), 1, 3.420874729116346e-05),
  (15, 0, 1, 8.890796518163113e-05),
  (16, (1, 0, 0), 1, 3.48480956982473e-05),
  (17, 0, 1, 2.0742464713512736e-05),
  (18, 0, 1, 2.516711361733543e-05),
  (19, (1, 0, 0), 1, 3.4427310326839896e-05),
  (20, 0, 1, 2.0982880619438946e-05),
  (21, (1, 0, 0), 1, 2.5303494825818905e-05),
  (22, 0, 1, 2.1872322378391143e-05),
  (23, 0, 1, 2.565451056971323e-05)],
 [(0, (3, 0, 2), 1, 0.0

In [ ]:
my_b = np.zeros(n_states)
my_b [0] = 1
derive_policy (my_b)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0,
  (0, 1, 0),
  0,
  (1, 0, 0),
  (0, 1, 0),
  (1, 0, 0),
  (1, 0, 0),
  0,
  (2, 2, 1),
  (1, 0, 0),
  0,
  (0, 0, 2),
  (1, 0, 0),
  0,
  (1, 0, 0),
  (2, 1, 0),
  (1, 0, 0),
  0,
  (1, 0, 0),
  0,
  0,
  0,
  (1, 0, 0),
  (1, 0, 0),
  0],
 [0.0,
  0.0,
  0.00010893609530967862,
  3.487299746961304e-05,
  2.0425787853701388e-05,
  0.00011866403095913024,
  2.3158256656782437e-05,
  1.6301394856832115e-05,
  2.3970002578375233e-05,
  0.0009091733635685081,
  7.432835329709889e-05,
  2.711761818486909e-05,
  0.0006316396556111712,
  2.6682353647531066e-05,
  2.454691910394963e-05,
  2.1643989508560555e-05,
  0.00013419452230554996,
  3.607380650248375e-05,
  2.7830416182763605e-05,
  2.161911268911623e-05,
  2.5455578739233565e-05,
  3.453631576589757e-05,
  3.3950748100298626e-05,
  2.1236084184344108e-05,
  1.655479759376069e-05])

In [ ]:

b_non_norm = np.random.rand(len(state_list))#[1,0,0,0,0,0]
#b_non_norm = np.zeros(n_states)
#b_non_norm [84] = 1
b_non_norm[(Tr_state[0])] = 0 ### treatment states: fully observable
b_non_norm[(Tr_state[1])] = 0 ### treatment states: fully observable
b_non_norm[(Tr_state[2])] = 0 ### treatment states: fully observable
b_non_norm[Death_state] = 0 ### death states: fully observablemy_b = np.zeros(n_states)
my_b = b_non_norm / np.sum(b_non_norm)
print("my_b", my_b)


my_b [0.01878656 0.01155315 0.01825157 0.00841453 0.00134776 0.01762099
 0.01525619 0.01852435 0.00960586 0.00414699 0.02044021 0.00945604
 0.01939244 0.01876593 0.01560176 0.01901367 0.01074781 0.01815264
 0.0030669  0.01450915 0.01421458 0.00429757 0.0030245  0.01209706
 0.01703063 0.00303139 0.00969104 0.018769   0.01360569 0.0064744
 0.00232643 0.00804103 0.01436777 0.01139371 0.01375415 0.01676034
 0.0026058  0.00939065 0.00626416 0.00964357 0.00705889 0.01001383
 0.00779217 0.01767888 0.00317302 0.01103057 0.00948103 0.01896627
 0.01017289 0.00085011 0.00689121 0.01312912 0.01928801 0.00474091
 0.0150041  0.00376535 0.01971294 0.0092473  0.00630398 0.00776922
 0.02115325 0.01230413 0.01929007 0.0054658  0.01952015 0.01438514
 0.00533825 0.00248415 0.01974202 0.01150393 0.02197545 0.00453435
 0.0111819  0.02145646 0.00149697 0.00587375 0.02073054 0.01290109
 0.00964729 0.00781314 0.01346027 0.00217773 0.01666875 0.00436497
 0.01970344 0.0142518  0.00906746 0.         0.         0.

In [ ]:
a_list, o_list, C_ris = derive_policy (my_b)

In [ ]:
for s in state_list:
    if (Q_val_PWLC[s,1,0] >= Q_val_PWLC[s,0,0]):
        print("C", key_list[s])
    else:
        print("W", key_list[s])
        

C 0
C (1, 0, 0)
C (0, 1, 0)
C (0, 0, 1)
C (2, 0, 0)
C (1, 1, 0)
C (0, 2, 0)
C (1, 0, 1)
C (0, 1, 1)
C (0, 0, 2)
C (3, 0, 0)
C (2, 1, 0)
C (1, 2, 0)
C (0, 3, 0)
C (2, 0, 1)
C (1, 1, 1)
C (0, 2, 1)
W (1, 0, 2)
C (0, 1, 2)
C (0, 0, 3)
C (4, 0, 0)
C (3, 1, 0)
C (2, 2, 0)
C (1, 3, 0)
C (0, 4, 0)
C (3, 0, 1)
C (2, 1, 1)
W (1, 2, 1)
W (0, 3, 1)
C (2, 0, 2)
C (1, 1, 2)
C (0, 2, 2)
C (1, 0, 3)
C (0, 1, 3)
C (0, 0, 4)
C (5, 0, 0)
C (4, 1, 0)
C (3, 2, 0)
W (2, 3, 0)
C (1, 4, 0)
C (0, 5, 0)
C (4, 0, 1)
C (3, 1, 1)
C (2, 2, 1)
W (1, 3, 1)
C (0, 4, 1)
W (3, 0, 2)
C (2, 1, 2)
C (1, 2, 2)
C (0, 3, 2)
C (2, 0, 3)
C (1, 1, 3)
C (0, 2, 3)
C (1, 0, 4)
C (0, 1, 4)
C (0, 0, 5)
W (6, 0, 0)
C (5, 1, 0)
C (4, 2, 0)
C (3, 3, 0)
W (2, 4, 0)
C (1, 5, 0)
C (0, 6, 0)
W (5, 0, 1)
C (4, 1, 1)
W (3, 2, 1)
C (2, 3, 1)
C (1, 4, 1)
C (0, 5, 1)
W (4, 0, 2)
C (3, 1, 2)
C (2, 2, 2)
C (1, 3, 2)
C (0, 4, 2)
C (3, 0, 3)
C (2, 1, 3)
C (1, 2, 3)
C (0, 3, 3)
C (2, 0, 4)
C (1, 1, 4)
C (0, 2, 4)
C (1, 0, 5)
C (0, 1, 5)
C (0, 0, 6)
